# This Jupyter Notebook is written to convert Raw Data and Scores files from NIH Toolbox IPAD exports into NDA data structures using linking information from a 'Crosswalk' and extra NDA-required subject identifier data (GUID, etc) from a csv.

Some Notes: 
using a specialty Python 3 virtual environment (named PycharmToolbox) as kernel for this notebook.
Installed by running the following commands in my terminal and then switching the kernel with the dropdown menu above:
> source /home/petra/.virtualenvs/PycharmToolbox/bin/activate
> pip install ipykernel
> ipython kernel install --user --name=PycharmToolbox
> jupyter-notebook

requirements file generated from within the activated virtual environment by:
> pip freeze > requirements.txt 


In [1]:
import os, datetime
import pandas as pd
import numpy as np
import subprocess

snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')


Specify the input and output data and paths for NIH toolbox.
To run the cells of this notebook, you will need four files.

Two are in the .csv format of the IPAD Toolbox applcation export.
E.g. a raw Data file containing scores for item level responses, and a Scores file, containing the summary statistics for the collection of item level data. We don't need the registration file, but it might be handy for filtering out batteries that were inappropriately administered.  These two files are linked by PIN and Inst variables, and must be cleaned a priori, to remove subjects that are in one but not the other file.  I.e. the list of unique PINs (ex. HCP0211999_V1) in one file should be exactly the same as the list of unique PINs in the other. For HCP data, we concatenate the exports of all subjects' Score data in to a single file, and the exports of all subjects Raw data into a second file.  Because all other sources of HCP data use 'subject' and 'visit' rather than a PIN which is a concatenation of both, we create these variables (subject and visit) from PIN prior to running this program as well.  

The third necessary file is a csv containing the fields that NDA requires in all of their structures 
e.g. subjectkey (GUID or pseudo-GUID), src_subject_id (e.g. HCP0211999), interview_age (in months), and gender (misnomer for sex assigned at birth).  In HCP data, we link the two sources of information via 'subject' and 'visit.'  

Lastly, read in the crosswalk file - which will map your vars to NDA after transpose is complete.  I have placed the crosswalk from HCPA as an example.  Any instruments in this crosswalk that are the same as yours (look at 'Inst' column) will work for you.  You will have to add any instruments not present, after obtaining variable maps and templates from the NDA for your particular set of NIH Toolbox Data.  

Note that subject and visit are variables we created locally to merge with the data coming from a different local source (REDCap).  They are not variables that are output from the NIH Toolbox app on the Ipads, but are necessary for the merge with the NDA required fields stored elsewhere.


In [2]:
#path for formatted structures
pathout="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/prepped_structures/"

In [3]:
#########################
#POINT TO YOUR DATA - a scores file and a raw file.  These will be the output from the IPAD, concatenated.  
# E.g. take all the scores and concatenate vertically in to a single file, and take all the raw item data and 
# concatenate into a single file.  Files are linked by PIN and Instrument.  
# In general there will be multiple row in the Raw Data file for every Instrument/PIN row in the Scores file, 
# All exceptions to this generality are accounted for below
##########################

#HCP has two different Lifespan studies going to NDA - HCP - Aging, and HCP Developement.  
#We have a few internal aliases for these studies that need to be labelled here; 
#hcp_studystr gets passed into the functions that label the output structures 
#ndar_studystr gets passed into the argument opens the csv that contains ALL of the HCP 
#subjects (they are related, so they have to be considered together) and subsets based on study
#csv scores and raw files for transformation - 


##########################
#hcp_studystr='yourstudy'
#ndar_studystr='placeholderforastringyouprobablywontneed'
#scoresD='pathtoyourscores/yourscores.csv'
#rawE='pathtoyourrawdata/yourraw.csv'
##########################

hcp_studystr='HCPD'
ndar_studystr='HCD'
scoresD='/home/petra/UbWinSharedSpace1/boxtemp/HCD_Toolbox_Scored_Combined_04_01_2020.csv'
rawD='/home/petra/UbWinSharedSpace1/boxtemp/HCD_Toolbox_Raw_Combined_04_01_2020.csv'

##########################
#hcp_studystr='HCPA'
#ndar_studystr='HCA'
#scoresD='/home/petra/UbWinSharedSpace1/boxtemp/HCAorBoth_Toolbox_Scored_Combined_04_01_2020.csv'
#rawD='/home/petra/UbWinSharedSpace1/boxtemp/HCAorBoth_Toolbox_Raw_Combined_04_01_2020.csv'



In [5]:
#read into dataframe and take a peak (we alphabetized our columns during concat process)
scordata=pd.read_csv(scoresD,header=0,low_memory=False)
scordata.head()

,Unnamed: 0,Age,Age-Corrected Standard Score,Age-Corrected Standard Scores Dominant,Age-Corrected Standard Scores Non-Dominant,AgeCorrCrystal,AgeCorrDCCS,AgeCorrEarly,AgeCorrEngRead,AgeCorrEngVocab,...,gender,iPad Version,pin,raw_cat_date,site,source,study,subject,v1_interview_date,visit
0,0,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,iPad Air 2 (WiFi),NaN,12_12_2019,NaN,NaN,NaN,HCD0033419,NaN,V1
1,1,NaN,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,iPad Air 2 (WiFi),NaN,12_12_2019,NaN,NaN,NaN,HCD0033419,NaN,V1
2,2,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,iPad Air 2 (WiFi),NaN,12_12_2019,NaN,NaN,NaN,HCD0033419,NaN,V1
3,3,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,iPad Air 2 (WiFi),NaN,12_12_2019,NaN,NaN,NaN,HCD0033419,NaN,V1
4,4,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,iPad Air 2 (WiFi),NaN,12_12_2019,NaN,NaN,NaN,HCD0033419,NaN,V1


In [7]:
rawdata=pd.read_csv(rawD,header=0,low_memory=False,error_bad_lines=True)
#rawdata.head()
rawdata.shape

(530585, 46)

In [8]:
#just taking a look here
rawdata.columns 

Index(['Unnamed: 0', 'App Version', 'Assessment Name', 'DataType',
       'DateCreated', 'DateCreatedDatetime', 'DeviceID', 'Firmware Version',
       'FirstDate4PIN', 'Inst', 'InstEnded', 'InstEndedDatetime', 'InstOrdr',
       'InstSctn', 'InstStarted', 'InstStartedDatetime', 'ItemID', 'ItmOrdr',
       'Locale', 'PIN', 'Position', 'Response', 'ResponseTime', 'SE', 'Score',
       'TScore', 'Theta', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'datediff', 'file_id', 'filename', 'flagged', 'gender', 'iPad Version',
       'index', 'level_0', 'parent', 'raw_cat_date', 'site', 'source', 'study',
       'subject', 'v1_interview_date', 'visit'],
      dtype='object')

In [9]:
rawdata.PIN.unique()[0:10]
#rawdata.groupby(['Inst','ItemID']).count()


array(['HCD0033419_V1', 'HCD0614237_V1', 'HCD0630134_V1', 'HCD0909761_V1',
       'HCD1005621_V1', 'HCD1123526_V1', 'HCD2134031_V1', 'HCD3023936_V1',
       'HCD3125035_V1', 'HCD3134743_V1'], dtype=object)

In [10]:
#prep the fields that NDA requires in all of their structures - we did this in another program, 
#and call it our ROSETTA STONE since output is required elsewhere.
#Here, just subsetting ROSETTA STONE to particular study (ndar_studystr='HCA' or 'HCD'), 
# renaming a few vars, and changing the date format
subjectlist='/home/petra/UbWinSharedSpace1/redcap2nda_Lifespan2019/Dev_pedigrees/UnrelatedHCAHCD_w_STG_Image_and_pseudo_GUID09_27_2019.csv'
subjects=pd.read_csv(subjectlist)[['subjectped','nda_gender', 'nda_guid', 'nda_interview_age', 'nda_interview_date']]
ndar=subjects.loc[subjects.subjectped.str.contains(ndar_studystr)].rename(
    columns={'nda_guid':'subjectkey','subjectped':'src_subject_id','nda_interview_age':'interview_age',
             'nda_interview_date':'interview_date','nda_gender':'gender'}).copy()
ndar['interview_date'] = pd.to_datetime(ndar['interview_date']).dt.strftime('%m/%d/%Y')
ndarlist=['subjectkey','src_subject_id','interview_age','interview_date','gender']


In [11]:
#this is the list of variables in the scored and raw data files that you might need...
#creating list in case your scored data is merged with other files for other reasons (ours was)
scorlist=['Age-Corrected Standard Score', 'Age-Corrected Standard Scores Dominant',
 'Age-Corrected Standard Scores Non-Dominant', 'AgeCorrCrystal', 'AgeCorrDCCS', 'AgeCorrEarly',
 'AgeCorrEngRead', 'AgeCorrEngVocab', 'AgeCorrFlanker', 'AgeCorrFluid', 'AgeCorrListSort',
 'AgeCorrPSM', 'AgeCorrPatternComp', 'AgeCorrTotal', 'Assessment Name', 'Computed Score',
 'ComputedDCCS', 'ComputedEngRead', 'ComputedEngVocab', 'ComputedFlanker', 'ComputedPSM',
 'ComputedPatternComp', 'DCCSaccuracy', 'DCCSreactiontime',  'Dominant Score', 'FlankerAccuracy',
 'FlankerReactionTime', 'FullTCrystal', 'FullTDCCS', 'FullTEarly', 'FullTEngRead', 'FullTEngVocab',
 'FullTFlanker', 'FullTFluid', 'FullTListSort', 'FullTPSM', 'FullTPatternComp', 'FullTTotal',
 'Fully-Corrected T-score', 'Fully-Corrected T-scores Dominant', 'Fully-Corrected T-scores Non-Dominant',
 'FullyCorrectedTscore', 'Group', 'Inst', 'InstrumentBreakoff', 'InstrumentRCReason', 'InstrumentRCReasonOther',
 'InstrumentStatus2', 'ItmCnt', 'Language', 'Male', 'National Percentile (age adjusted)',
 'National Percentile (age adjusted) Dominant', 'National Percentile (age adjusted) Non-Dominant',
 'Non-Dominant Score', 'PIN', 'Raw Score Left Ear', 'Raw Score Right Ear', 'RawDCCS',
 'RawFlanker', 'RawListSort', 'RawPSM', 'RawPatternComp', 'RawScore', 'SE', 'Static Visual Acuity Snellen',
 'Static Visual Acuity logMAR', 'TScore', 'Theta', 'ThetaEngRead', 'ThetaEngVocab', 'ThetaPSM', 'Threshold Left Ear',
 'Threshold Right Ear', 'UncorrCrystal', 'UncorrDCCS', 'UncorrEarly', 'UncorrEngRead', 'UncorrEngVocab',
 'UncorrFlanker', 'UncorrFluid', 'UncorrListSort', 'UncorrPSM', 'UncorrPatternComp', 'UncorrTotal',
 'Uncorrected Standard Score', 'Uncorrected Standard Scores Dominant', 'Uncorrected Standard Scores Non-Dominant',
 'UncorrectedStandardScore']
rawlist=['App Version', 'Assessment Name', 'DataType','DateCreated', 'DeviceID',  'Firmware Version',  
 'Inst', 'InstEnded','InstEndedDatetime', 'InstOrdr', 'InstSctn', 'InstStarted','InstStartedDatetime',
 'ItemID', 'ItmOrdr', 'Locale','PIN', 'Position', 'Response', 'ResponseTime', 'SE', 'Score', 'TScore',
 'Theta','iPad Version']

In [12]:
#merge the score and raw data with the required fields for the NDA
#Note that subject and visit are HCP specific variables that we use to subset the records being sent to the NDA
#Depending on how you organized your data, you may need to create dummy vars if you dont have them...
#scordata['subject']=scordata.PIN #or some other variable in scordata that can be used to merge with ndarfields data
#scordata['visit']='V1' #we keep this around because eventually we'll be releaseing V2,V3, and FU data
#rawdata['subject']=rawdata.PIN
#rawdata['visit']='V1'

scordata=pd.merge(scordata[scorlist+['subject','visit']],ndar,how='inner',left_on='subject', right_on='src_subject_id')
rawdata=pd.merge(rawdata[rawlist+['subject','visit']],ndar,how='inner',left_on='subject', right_on='src_subject_id')


Do a little QC and data exploration

In [13]:
#NOW that you have ALL of your data, its helpful to remember what the goal output is going to look like.  

#first look to see what sort of Instrument overlap you have. For example, Cognition Composite Instruments only 
#show up in the scores files, where as most other instruments show up in both.
#this is relevant to how the data is transformed.  

#check that your instruments are in both raw data and scores files...if not, you may need special coding blocks
#For instruments in Raw but not scores, HCP has practices (not sent to NDA - ignore), 
#and NIH Toolbox Pain Intensity FF Age 18+ v2.0 Instrument (requires special coding - see below)

#For instruments in Scores but not Raw, 
#Cognition Composites (requires special coding - see below), 
#Then there are the other Summary scores that should be handled by the code below
#Negative Affect Summary (18+)
#Social Satisfaction Summary (18+)
#Psychological Well Being Summary (18+)

#ignore the Emo Instructions...not being sent

#check your data and adjust if needed 
print('*****Instruments in Raw data but not Scores:')
for i in rawdata.Inst.unique():
    if i not in scordata.Inst.unique():
        print(i)
print('******Instruments in Scored data but not Raw:')
for i in scordata.Inst.unique():
    if i not in rawdata.Inst.unique():
        print(i)

#one of the instruments, NIH Toolbox List Sorting Working Memory Test Ages 3-6 v2.1, is showing up for protocol
#deviation reasons in our data (someone opened and closed a battery...lookslike.  The validated column of the crosswalk has been set to NO for this instrument.  If more data make
#this tool necessary, you'll have to resolve why its showing up in scores and not items (if it still does)
#nan rows in Raw data correspond to cases where Registration Data was uploaded as 'raw' data somewhere.
#you will need to QC (we just allow program to drop them here - send flag back to data curation team for next release)

*****Instruments in Raw data but not Scores:
NIH Toolbox Pattern Comparison Processing Speed Test Age 7+ Practice v2.1
NIH Toolbox Pain Intensity FF Age 18+ v2.0
******Instruments in Scored data but not Raw:
Cognition Fluid Composite v1.1
Cognition Crystallized Composite v1.1
Cognition Total Composite Score v1.1
Cognition Early Childhood Composite v1.1
NIH Toolbox Visual Acuity Practice Age 8+ v2.0
Negative Affect Summary (18+)
Social Satisfaction Summary (18+)
Psychological Well Being Summary (18+)
Negative Affect Summary (8-12)
Social Satisfaction Summary (8-12)
Negative Social Perception Summary (8-12)
Psychological Well Being Summary (8-12)
Negative Affect Summary (13-17)
Social Satisfaction Summary (13-17)
Negative Social Perception Summary (13-17)
Psychological Well Being Summary (13-17)
Negative Peer Relations Parent Report Summary (8-12)
Psychological Well Being Parent Report Summary (8-12)
NIH Toolbox List Sorting Working Memory Test Ages 3-6 v2.1


In [14]:
#check that lengths are the same...indicating one to one PIN match between scores and raw
print(len(rawdata.PIN.unique()))
print(len(scordata.PIN.unique()))
#check that shape is same before and after removing duplicates (should not be any)
rawdata.shape
scordata.shape
print(rawdata.shape)
print(scordata.shape)
testraw=rawdata.drop_duplicates(subset={'PIN','Inst','ItemID','Position'},keep='first')
testscore=scordata.drop_duplicates(subset={'PIN','Inst'})
print(testraw.shape)
print(testscore.shape)


656
656
(233432, 32)
(25645, 96)
(233432, 32)
(25645, 96)


In [15]:
#define the function that will turn a prepared (e.g already transformed, renamed, revalued, etc. and otherwise
# ready to go) dataframe into a csv structure 
def data2struct(patho,dout,crosssub,study):
    """
    Convert dout, a prepared pandas dataframe, into a csv structure that NDA can import
    
    parameters: 
    patho - full path to place you want to store structures (there will be many)
    dout - name of data frame that contains all the variables to be exported
    crosssub - a dataframe which is the subset of the crosswalk for the instrument to be exported as structure
    study - a string to put in the name of the csv file along with the structure name and the short name of the instrument
    
    note that snapshotdate is globally defined external to this funtion near import statments...     
    
    """
    #get the name and number of the structure from the crosswalk subset
    strucroot=crosssub['nda_structure'].str.strip().str[:-2][0]
    strucnum=crosssub['nda_structure'].str.strip().str[-2:][0]
    #prepare the name of the output file and path
    instshort=crosssub['inst_short'].str.strip()[0]
    inst=crosssub['Inst'].str.strip()[0].replace(' ','_').replace('+','plus').replace('-','_') 
    filePath=os.path.join(pathout,study+'_'+instshort+'_'+strucroot+strucnum+'_'+snapshotdate+'.csv')
    if os.path.exists(filePath):
        os.remove(filePath)
    else:
        pass
        #print("Can not delete the file as it doesn't exists")
    with open(filePath,'a') as f:
        f.write(strucroot+","+str(int(strucnum))+"\n")
        dout.to_csv(f,index=False)


In [17]:
#define the function that can be used for the instruments that follow more generalizable pattern
#This function will alert you to any instruments that were successfully transformed but tha might
#warrent a closer look.
#function takes a dataframe and formats the column names such that all the special characters are removed 
#the export has characters that python and the NDA dont like
#then for a given istrument, find the rows in the crosswalk corresponding to this instrument.  
def sendthroughcrosswalk(pathout,instreshapedfull,inst_i,crosswalk,studystr,verbose,debug):
    """
    Send instreshapedfull, a dataframe that has pivoted the item level data into the scored data by instrument
    through the crosswalk to have its variables renamed and reformatted according to the harmonization 
    requests of the NDA
    
    parameters: 
    pathout - full path to place you want to store structures (there will be many) - argument gets passed to data2struct fuctnion
    inst_i - string name of instrument as it appears in the NIH Toolbox output, exactly (case sensitive with version)
    crosswalk - pandas dataframe of crosswalk (read from csv)
    studystr - 'HCPA' or other string specified at the beginning of this notebook - will be passed to data2struct funciton to tag the file
               name with the study source of the data
    verbose - YES or NO, will flag all the variable to element mappings available in the crosswalk 
              that weren't called upon in this transformation because they weren't found in your data
    debug - YES or NO, will print out the last row to be executed from the 'requested_python' column, in case you get an 
            Error, and need to figure out where the loop got stuck
    """
    # replace special charaters in column names
    instreshapedfull.columns = instreshapedfull.columns.str.replace(' ', '_').str.replace('-', '_').str.replace('(','_').str.replace(')', '_')
    crosswalk_subset = crosswalk.loc[crosswalk['Inst'] == inst_i]
    crosswalk_subset.reset_index(inplace=True)
    #
    if crosswalk_subset.reset_index().validated[0]=='NO':
            print("Skipping "+inst_i+ " because crosswalk not yet validated for this instrument ")
    else: 
        # some studies will have some but not all of the variables in hcp_variable (result of skip logic, perhaps)
        # need to make sure they know about this(in case not due to skip logic) but also that code is only execute for
        # vars in existence.  
        #also need to keep track of dummy vars that dont exist in IPAD output, but are necessary for NDA and need to be created 
        #on the fly
        #how many vars are in the instrument according to what is stored in hcp_variable? 
        cwlistbef = list(crosswalk_subset['hcp_variable'])
        before = len(cwlistbef)
        #how many are in the intersection of hcp_variable and the prepared data (e.g. what shows up with the particular instrument in instreshapedfull)
        cwlist = list(set(cwlistbef) & set(
            instreshapedfull.columns))  # drop the handful of vars in larger instruments that got mapped but that we dont have
        after = len(cwlist)
        if before != after:
            print("WARNING!!! " + inst_i + ": Crosswalk expects " + str(before) + " elements, but only found " + str(after)+ " in the prepared data")
            notfound=list(np.setdiff1d(cwlistbef,cwlist))
            if verbose=='YES':
                print("Not Found:"+ str(notfound))
        #get the dummies
        dummys=[]
        for i in cwlistbef:
            if "dummy" in i:
                dummys=dummys+[i]
        #studydata should have all the ndar variables and the list of vars in the intersection of what exists in ipad output and
        #what can be received per the crosswalk.  
        # if you dont force the intersection then code will try to execute on things that dont exist
        #The new dummy variables will be created on the fly.
        studydata = instreshapedfull[ndarlist + cwlist].copy()
        # execute any python one liners (see how they all refer to studydata?) for all rows in the crosswalk corresponding to cwlist this instrument vars except the notfounds.
        itersubset=crosswalk_subset.loc[crosswalk_subset.hcp_variable.isin(cwlist + dummys)]
        for index, row in itersubset.iterrows():#crosswalk_subset.iterrows():
            if pd.isna(row['requested_python']):
                pass
            else:
                if debug=='YES':
                    print(row['requested_python'])
                exec(row['requested_python'])
        uploadlist = list(crosswalk_subset['hcp_variable_upload'])
        uploadlist = list(set(uploadlist) & set(studydata.columns))
        data2struct(patho=pathout, dout=studydata[ndarlist + uploadlist], crosssub=crosswalk_subset, study=studystr)

In [18]:
#specify your crosswalk- take a peak - use the latest crosswalk from the https://github.com/humanconnectome/NIHToolbox2NDA/
#e.g. Crosswalk_NIH_Toolbox_2_NDA.csv
crosswalkpath="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/"
cfile="Crosswalk_NIH_Toolbox_2_NDA.csv"
crosswalk=pd.read_csv(crosswalkpath+cfile,header=0,low_memory=False, encoding = "ISO-8859-1")
crosswalk.head()


,validated,Measurement System,Domain,template,inst_short,Item ID,Stem,Context,DataType,Responses,...,action_requested,requested_python,specialty_code,hcp_variable_upload,nda_structure,nda_element,description,valueRange,notes,Source
0,NaN,NaN,NaN,NaN,Anxiety_Summary_3-7,NaN,NaN,NaN,NaN,NaN,...,rename to version_form,studydata['version_form']=studydata['Inst'],NaN,version_form,tlbx_fearanx01,version_form,Form used/assessment name,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Anxiety_Summary_3-7,NaN,NaN,NaN,NaN,NaN,...,"strip 'Assessment' from text, and rename to fn...",studydata['fneproc']=studydata['Assessment_Nam...,NaN,fneproc,tlbx_fearanx01,fneproc,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Anxiety_Summary_3-7,NaN,NaN,NaN,NaN,NaN,...,rename to comqother,studydata['comqother']=studydata['respondent'],NaN,comqother,tlbx_fearanx01,comqother,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,NaN,NaN,NaN,NaN,NaN,...,NaN,studydata['nih_tlbx_fctsc']=studydata['Fully_C...,NaN,nih_tlbx_fctsc,tlbx_fearanx01,nih_tlbx_fctsc,Fully-Corrected T-Score,0::120,NaN,HCPD
4,NaN,NaN,NaN,Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,NaN,NaN,NaN,NaN,NaN,...,Please rename 'Language' to 'primary_language'...,studydata['primary_language']=studydata['Langu...,NaN,primary_language,tlbx_fearanx01,primary_language,Subject's Primary Language,NaN,NaN,HCPD


In [20]:
#For some instruments we only want to send scores.  See discusssion about Visual Acuity Instruements below, for example. 
#For other instruments we only CAN send scores, either because only score level data is avaialbe for this 
# instrument (see next code block)
# or because only score level data is ready to go (item level data not yet mapped at the NDA)
#for the PSMT instruments that arent validating..were waiting on a missing code from the NDA (e.g 999) to 
#use for the required variables.  

scoresonly=scordata.loc[(scordata.Inst.str.contains('Visual Acuity')==True) | (scordata.Inst.str.contains('Picture Sequence Memory Test')==True)]
for i in scoresonly.Inst.unique():
    inst_i=i
    if "Practice" in inst_i:
        print("Note:  Omitting practice instrument, "+inst_i)
    else:    
        try:  #this will fail if there are duplicates or if no-one has the data of interest (e.g. idlist too small), or if only V2 instrument
            #print('Processing '+inst_i+'...')
            instreshapedfull=scordata.loc[scordata.Inst==inst_i][scorlist+ndarlist]
            instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
            #verbose will tell you which items were not found, debug will print out the python lines before they are executed
            if 'Parent' in inst_i:
                instreshapedfull['respondent']='parent'
            else:
                instreshapedfull['respondent']='self'
            sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='NO',debug='YES')
        except:
            print('Couldnt process '+inst_i+'...')



Skipping NIH Toolbox Picture Sequence Memory Test Age 8+ Form A v2.1 because crosswalk not yet validated for this instrument 
Note:  Omitting practice instrument, NIH Toolbox Visual Acuity Practice Age 8+ v2.0
WARNING!!! NIH Toolbox Visual Acuity Test Age 8+ v2.0: Crosswalk expects 112 elements, but only found 12 in the prepared data
studydata['nih_tlbx_agegencsc']='999'
studydata['nih_tlbx_rawscore']='999'
studydata['nih_tlbx_se']='999'
studydata['nih_tlbx_theta']='999'
studydata['nih_tlbx_tscore']='999'
studydata['natl_percentile__age_adjusted_']=studydata['National_Percentile__age_adjusted_'].fillna(-9999).astype(int).astype(str).str.replace('-9999','')
studydata['age_corrected_standard_score']=studydata['Age_Corrected_Standard_Score'].fillna(-9999).astype(int).astype(str).str.replace('-9999','')
studydata['nih_tlbx_fctsc']=studydata['Fully_Corrected_T_score']
studydata['version_form']=studydata['Inst']
studydata['fneproc']=studydata['Assessment_Name'].str.replace('Assessment ','')


Do special cases last

In [22]:
#For cases where instrument is in the scored data but not the raw data, e.g. 
# because this is a summary across instruments, is instructions/practice or because someone 
# opened and closes a battery before generating any item level data 

for i in scordata.Inst.unique():
    if i not in rawdata.Inst.unique():
        inst_i=i
        if "Cognition" in inst_i:
            pass  #special case--see specialty code block below
        elif "Practice" in inst_i:
            print("Note:  Omitting practice instrument, "+inst_i)
        elif "Instructions" in inst_i:
            print("Note:  Omitting Instructions instrument, "+inst_i)
        else:
            try:  #this will fail if there are duplicates or if no-one has the data of interest (e.g. idlist too small), or if only V2 instrument
                #print('Processing '+inst_i+'...')
                instreshapedfull=scordata.loc[scordata.Inst==inst_i][scorlist+ndarlist]
                instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
                #verbose will tell you which items were not found, debug will print out the python lines before they are executed
                if 'Parent' in inst_i:
                    instreshapedfull['respondent']='parent'
                else:
                    instreshapedfull['respondent']='self'
                sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='YES',debug='No')
            except:
                print('Couldnt process '+inst_i+'...')

#Lots of dummy variables were needed to fit the IPAD data into the NDA structures without needing a special code block for everye
#single structure (see cogcomp01 specialty code below).  The reason for this is that many of these summary scores were coming 
#from different instruments before the instruments (NIH Toolbox term) were defined and versione and merged into the 
#same structure.  

Note:  Omitting practice instrument, NIH Toolbox Visual Acuity Practice Age 8+ v2.0
WARNING!!! Negative Affect Summary (18+): Crosswalk expects 8 elements, but only found 5 in the prepared data
Not Found:['dummy_00001', 'dummy_00002', 'dummy_00003']
WARNING!!! Psychological Well Being Summary (18+): Crosswalk expects 8 elements, but only found 5 in the prepared data
Not Found:['dummy_00035', 'dummy_00036', 'dummy_00037']
WARNING!!! Negative Affect Summary (8-12): Crosswalk expects 9 elements, but only found 5 in the prepared data
Not Found:['dummy_00050', 'dummy_00051', 'dummy_00052', 'dummy_00053']
WARNING!!! Negative Social Perception Summary (8-12): Crosswalk expects 10 elements, but only found 6 in the prepared data
Not Found:['dummy_00061', 'dummy_00062', 'dummy_00063', 'dummy_00064']
WARNING!!! Psychological Well Being Summary (8-12): Crosswalk expects 9 elements, but only found 5 in the prepared data
Not Found:['dummy_00091', 'dummy_00092', 'dummy_00093', 'dummy_00094']
WARNING!

In [24]:
#for non-special instruments in both scores AND raw data types (skip the scoresonly ones from above)
#add a check to make sure that everything found in the data has a row in the crosswalk
#add indicator for whether this is a variable from the scores file or a variable from the raw data file.
for i in scordata.Inst.unique():
    if i in rawdata.Inst.unique():
        inst_i=i
        if "Visual Acuity" in inst_i:
            pass  #special case--see below
        elif "Practice" in inst_i:
            print("Note:  Omitting practice instrument, "+inst_i)
        else:
            try:  #this will fail if there are duplicates or if no-one has the data of interest (e.g. idlist too small), or if only V2 instrument
                #print('Processing '+inst_i+'...')
                items=rawdata.loc[rawdata.Inst==inst_i][['PIN','subject','Inst','visit','ItemID','Position',
                   'subjectkey','src_subject_id','interview_age','interview_date','gender',
                   'Score','Response','ResponseTime']]# not these..., 'SE', 'Score', 'TScore','Theta']]
                items.ItemID = items.ItemID.str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_').str.replace(' ','_')
                inst=items.pivot(index='PIN',columns='ItemID',values='Score').reset_index()
                meta=items.drop_duplicates(subset=['PIN','visit'])
                instreshaped = pd.merge(meta, inst, on='PIN', how='inner').drop(columns={'subject', 'visit','Inst'})
                items2=scordata.loc[scordata.Inst==inst_i][scorlist]
                instreshapedfull=pd.merge(instreshaped,items2,on='PIN',how='inner')
                instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
                #verbose will tell you which items were not found, debug will print out the python lines before they are executed
                if 'Parent' in inst_i:
                    instreshapedfull['respondent']='parent'
                else:
                    instreshapedfull['respondent']='self'
                sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='YES',debug='NO')
            except:
                print('ERROR: Couldnt process '+inst_i+'...')

#NOTE: its okay if there are items in the crosswalk that doent exist in your data...lots of reasons
#not okay if there are scores in the crosswalk that doent exsit in the data...need to investigate
#also will be flags for dummy vars because they exist in crosswalk but not data (they were created for the NDA)
#turn verbose off to ignore these warnings.  Errors will still be reported.
# for instruments not validated--check crosswalk. v2.1 of some isntruments dont exist in the NIH Toolbox Data
# Dictionary yet, so its pointless to try to pretend that we know what they map to in the NDA.  

WARNING!!! NIH Toolbox Flanker Inhibitory Control and Attention Test Age 12+ v2.1: Crosswalk expects 38 elements, but only found 34 in the prepared data
Not Found:['flanker_arrow_prac5', 'flanker_arrow_prac6', 'flanker_arrow_prac7', 'flanker_arrow_prac8']
WARNING!!! NIH Toolbox List Sorting Working Memory Test Age 7+ v2.1: Crosswalk expects 37 elements, but only found 36 in the prepared data
Not Found:['wmls1l2t1']
WARNING!!! NIH Toolbox Dimensional Change Card Sort Test Age 12+ v2.1: Crosswalk expects 52 elements, but only found 48 in the prepared data
Not Found:['dccs_color_prac5', 'dccs_color_prac6', 'dccs_color_prac7', 'dccs_color_prac8']
Skipping NIH Toolbox Pattern Comparison Processing Speed Test Age 7+ v2.1 because crosswalk not yet validated for this instrument 
Skipping NIH Toolbox Picture Sequence Memory Test Age 8+ Form A v2.1 because crosswalk not yet validated for this instrument 
WARNING!!! NIH Toolbox Positive Affect CAT Age 18+ v2.0: Crosswalk expects 30 elements, but 

In [90]:
##test area for testing individual instruments
inst_i='NIH Toolbox Picture Sequence Memory Test Age 8+ Form A v2.1'
items=rawdata.loc[rawdata.Inst==inst_i][['PIN','subject','Inst','visit','ItemID','Position',
   'subjectkey','src_subject_id','interview_age','interview_date','gender',
   'Score','ResponseTime']]# not these..., 'SE', 'Score', 'TScore','Theta']]
items.ItemID = items.ItemID.str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_').str.replace(' ','_')
inst=items.pivot(index='PIN',columns='ItemID',values='Score').reset_index()
meta=items.drop_duplicates(subset=['PIN','visit'])
instreshaped = pd.merge(meta, inst, on='PIN', how='inner').drop(columns={'subject', 'visit','Inst'})
items2=scordata.loc[scordata.Inst==inst_i][scorlist]

instreshapedfull=pd.merge(instreshaped,items2,on='PIN',how='inner')
instreshapedfull['version_monster']=instreshapedfull['Inst']+','+instreshapedfull['Assessment Name']
if 'Parent' in inst_i:
    instreshapedfull['respondent']='parent'
else:
    instreshapedfull['respondent']='self'
sendthroughcrosswalk(pathout,instreshapedfull, inst_i, crosswalk,studystr=hcp_studystr,verbose='No',debug='Yes')
#for i in instreshapedfull.columns:
#    print(i)
#meta.columns
##items2.columns

studydata['nih_picseq_ageadjusted']=studydata['Age_Corrected_Standard_Score']; studydata.loc[studydata.nih_picseq_ageadjusted.isnull()==True,'nih_picseq_ageadjusted']='999'
studydata['version_form']=studydata['Inst']
studydata['fneproc']=studydata['Assessment_Name'].str.replace('Assessment ','')
studydata['comqother']=studydata['respondent']
studydata['nih_picseq_computed']=studydata['Computed_Score']; studydata.loc[studydata.nih_picseq_computed.isnull()==True,'nih_picseq_computed']='999'
studydata['wcst_ni']=studydata.ItmCnt.round().fillna(-99).astype(int).astype(str).str.replace('-99','')
studydata['interview_language']=studydata['Language']
studydata['nih_picseq_natperc']=studydata['National_Percentile__age_adjusted_']; studydata.loc[studydata.nih_picseq_natperc.isnull()==True,'nih_picseq_natperc']='999'
studydata['nih_picseq_raw']=studydata.RawScore.round().fillna(999).astype(int).astype(str)
studydata['nih_tlbx_se']=studydata['SE']
studydata['nih_tlbx_theta']=studydata['Theta']
st

In [25]:
#special coding required for instruments in the crosswalk that have jupyter in their specialty code columns
#Within the rawdata structure (for HCP), all but the NIH Toolbox Pain Intensity FF Age 18+ v2.0 Instrument are practices
#So only the Pain Intensity instrument needed special coding attention 
#check your data and adjust if needed - note that subject and visit are variables we created locally 
#to merge with the data coming from a different local source (REDCap)
#create the NDA structure for this special case
#this structure doesnt get 'sent through crosswalk' so any code that is in python column wont get executed

inst_i='NIH Toolbox Pain Intensity FF Age 18+ v2.0'
#most of the rows contain duplicated information...only need to know the PIN once, for example, not once for each item response
# so values in the response column need to be pivoted and then merged with the rest of the data, 
paindata=rawdata.loc[rawdata.Inst==inst_i][['PIN','subject','Inst','visit','ItemID','Position',
        'subjectkey','src_subject_id','interview_age','interview_date','gender',
        'Response','ResponseTime', 'SE', 'Score', 'TScore','Theta','Assessment Name']]
paindata.ItemID = paindata.ItemID.str.lower().str.replace('-','_').str.replace('(','_').str.replace(')','_')
inst = paindata.pivot(index='PIN', columns='ItemID', values='Response').reset_index()
meta = paindata.drop_duplicates(subset=['PIN', 'visit'])

#meta['Inst']=inst_i
painreshaped = pd.merge(meta, inst, on='PIN', how='inner').drop(columns={'subject','visit','PIN'})
crosswalk_subset=crosswalk.loc[crosswalk['Inst']==inst_i]
crosswalk_subset.reset_index(inplace=True)
cwlist=list(crosswalk_subset['hcp_variable_upload']) #these should all correspond with the nda_element names in this structure

#several dummy vars for required vars - normally these would have placeholders in the meta (scores) files but 
#since this particular instrument only exists in the raw data, we have to explicitly create place holders
painreshaped['nih_tlbx_agegencsc']=999
painreshaped['nih_tlbx_rawscore']=999
painreshaped['nih_tlbx_tscore']=999
painreshaped['nih_tlbx_se']=999
painreshaped['nih_tlbx_theta']=999
painreshaped['respondent']='self'
painreshaped['version_form']=painreshaped['Inst']
painreshaped['fneproc']=painreshaped['Assessment Name'].str.replace('Assessment ','')
painreshaped['comqother']=painreshaped['respondent']

#painreshaped['version_form']=painreshaped.Inst
#+','+painreshaped['Assessment Name']

reshapedslim=painreshaped[ndarlist+cwlist]

#the data2struct function only uses the crosswalk to get the structure name and number for the header of dout
#dout is otherwise ready to go and data2structure just writes it to a file in the specified location
data2struct(patho=pathout,dout=reshapedslim,crosssub=crosswalk_subset,study=hcp_studystr)

In [26]:
# Another special case is for Cognition Composite scores all v1.1 - going to cogcomp01 structure at the NDA- 
# Cog comp is special for several reason...it doesnt have corresponding entries in the raw data because
# it represents a summary across instruments in the Cognitive domain.  Even so, 4 cog comp 'instruments' are going to 
# a single NDA structure.  Each of these insturments has a version number and an assessmen
# This was mapped before Leo agreed to accept data by NIH Toolbox Instrument name (pivot by Inst)
# keeping this special case coding in for posterity and to shed light on one type of merge he must do on his end
# and the fact that this special situation is not yet being addressed (unless they can take multiple rows per person)
# Note that this structure illustrates the versioning problem when merging several NIH Toolbox Instruments together
# onto the same row
# Instruments are being mapped to the same # buckets
# One of the main issues that will hopefully be resolved by teleconference 3/23 is how the NDA is keeping track of 
# several instruments (with different versions) getting mapped to the same rows in a structure
# when it comes to NIH toolbox data

cogcompdata=scordata.loc[scordata.Inst.str.contains('Cognition')==True][['PIN','Language',
    'Assessment Name','Inst',  'Uncorrected Standard Score', 'Age-Corrected Standard Score',
    'National Percentile (age adjusted)', 'Fully-Corrected T-score']+ndarlist]

#initialize prefix
cogcompdata['varprefix']='test'
cogcompdata.loc[cogcompdata.Inst=='Cognition Crystallized Composite v1.1','varprefix']='nih_crystalcogcomp_'
cogcompdata.loc[cogcompdata.Inst=='Cognition Early Childhood Composite v1.1','varprefix']='nih_eccogcomp_'
cogcompdata.loc[cogcompdata.Inst=='Cognition Fluid Composite v1.1','varprefix']='nih_fluidcogcomp_'
cogcompdata.loc[cogcompdata.Inst=='Cognition Total Composite Score v1.1','varprefix']='nih_totalcogcomp_'

#pivot the vars of interest by varprefix and rename
uncorr=cogcompdata.pivot(index='PIN',columns='varprefix',values='Uncorrected Standard Score')
for col in uncorr.columns.values:
    uncorr=uncorr.rename(columns={col:col+"unadjusted"})
ageadj=cogcompdata.pivot(index='PIN',columns='varprefix',values='Age-Corrected Standard Score')
for col in ageadj.columns.values:
    ageadj=ageadj.rename(columns={col:col+"ageadj"})
npage=cogcompdata.pivot(index='PIN',columns='varprefix',values='National Percentile (age adjusted)')
for col in npage.columns.values:
    npage=npage.rename(columns={col:col+"np_ageadj"})

#put them together
cogcompreshape=pd.concat([uncorr,ageadj,npage],axis=1)

#hijacking what is the same for all four instruments
meta=cogcompdata[['PIN','Language']+ndarlist].drop_duplicates(subset={'PIN'})

#all the data in place
cogcompreshape=pd.merge(meta,cogcompreshape,on='PIN',how='inner')

# Now grabbing version and assessment info for version_form
# initial attempt to capture the version failed ...they got mapped to raw scores 
# and failed validation 
# per email, all of this information will go to the 'version_form' variable.
meta2=cogcompdata[['PIN','Inst','Assessment Name']].drop_duplicates(subset={'PIN','Inst'})
meta2['Inst,Assessment Name']=meta2['Inst']+','+meta2['Assessment Name']
meta3=meta2.pivot(index='PIN',columns='Inst',values='Inst,Assessment Name')
#this will only work until there are more than one versions of the composites in the data
#need to make it more flexible so that there are 4 possible instruments (whatever version they may be)
meta3['version_monster']=meta3['Cognition Crystallized Composite v1.1']+';'+meta3['Cognition Early Childhood Composite v1.1']+';'+meta3['Cognition Fluid Composite v1.1']+';'+meta3['Cognition Total Composite Score v1.1']

meta3['nih_crystalcogcomp']=meta3['Cognition Crystallized Composite v1.1']
meta3['nih_eccogcomp']=meta3['Cognition Early Childhood Composite v1.1']
meta3['nih_fluidcogcomp']=meta3['Cognition Fluid Composite v1.1']
meta3['nih_totalcogcomp']=meta3['Cognition Total Composite Score v1.1']

##for i in meta3.columns.to_list();  
#meta3=meta3['version_form'].reset_index()

cogcompreshape=pd.merge(cogcompreshape,meta3,on='PIN',how='inner')
#cogcompreshape.columns
inst_i='Cognition Composite Scores'  #one instrument here...is merging of four instruments there

#crosswalk_subset=crosswalk.loc[crosswalk.Inst==inst_i]
#cwlist=list(crosswalk_subset['hcp_variable_upload']) #these should all correspond with the nda_element names in this structure
#reshapedslim=cogcompreshape[ndarlist+cwlist]
#cogcompreshape[cwlist]
#the data2struct function only uses the crosswalk to get the structure name and number for the header of dout
#dout is otherwise ready to go and data2structure just writes it to a file in the specified location
#data2struct(patho=pathout,dout=reshapedslim,crosssub=crosswalk_subset,study=hcp_studystr)


sendthroughcrosswalk(pathout,cogcompreshape,inst_i,crosswalk,studystr=hcp_studystr,verbose='No',debug='Yes')

In [51]:
#Visual Acuity instruments, for example, have an unknown and variable number of 
#repeated items at different 'positions' which would require a double transpose into a single instrument 'row' per person.
#NDA mapped all the different positions we saw in our data (see placeholders in crosswalk), but will assuredly 
#not have all of your positions.  For visual acuity, the scores matter more than the individual items, however, 
#so we opted to only send scores for this particular instrument.  Feel free to extend the specialty code below to 
#accommodate item level instrument for visual acuity.  
#Visual Acuity items not yet mapped - come back to it if possible before the release otherwise omit item levels this release
#scores are mapped in the first special case above.


#Last special Case is for Visual Acuity, which needs double pivot because of repeat items at different positions
#This special case not yet mapped by NDA - so don't run, but will look something like this
#special case for instruments with "Visual Acuity" in their titles, which have dup inst/itemid at diff positions
#for i in scordata.Inst.unique():
#    if i in rawdata.Inst.unique():
#        inst_i=i
#        if "Visual Acuity" in inst_i:
#            print('Processing ' + inst_i + '...')
#                items=rawdata.loc[rawdata.Inst.str.contains('Visual Acuity')][['PIN','subject','Inst',
#                   'gender','visit','ItemID','Position','Response']]
#                items.ItemID = items.ItemID.str.lower()
#               items['dup_number']=items.groupby(['PIN','ItemID']).cumcount()+1
#               items['ItemID_Dup']=items.ItemID.str.replace('|', '_') + '_P'+items.dup_number.astype(str)
#               inst=items.pivot(index='PIN',columns='ItemID_Dup',values='Response')
#               meta = items.drop_duplicates(subset=['PIN', 'visit'])[['Inst', 'PIN', 
#                                                              'subject', 'visit']]
#               instreshaped = pd.merge(meta, inst, on='PIN', how='inner')
#               items2 = scordata.loc[scordata.Inst == inst_i]
#               instreshapedfull = pd.merge(instreshaped, items2, on='PIN', how='inner')

now validate all of these files by calling the OS from within this notebook (assuming you are using linux) to run the NDA validator on your command line.  Alternatively, you could just navigate to your terminal and
execute the following for loop .  

for var in pathout/*.csv; do vtcmd $var; done

Either option requires that you have downloaded and installed https://github.com/NDAR/nda-tools python package
per instructions.  I installed vtcmd in my home directory, which set a couple defaults in place., such as the location of validation results. To have the output of the validation sent to a more meaningful location than than the default, I opened the /home/petra/.NDATools/settings.cfg file, and  
changed the line under [Files] that says 'validation_results = NDAValidationResults' to a better place (perhaps 'pathout').  Example, mine now says 
validation_results = /home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/NDAValidationResults

so that the prepped structures directory and the NDAValidationResults Directory are right next to one another.


If you had an error in the validation, your likely course of action is to debug the python code in the the crosswalk. Here are some unix commands to help filter through common issues in the Validation results:  

Find all the not integer warnings:
grep notInteger /home/petra/NDAValidationResults/* > Notintegerwarnings

Find all the invalid range warnings 
grep "invalid" NDAValidationResults/* | cut -d ',' -f 1,6

Cat all of the validation results together so you can see them all at once
cat NDAValidationResults/validation*.csv > NDAValidationResults/Allvalidations.csv 